In [1]:
import pandas as pd
import os
import numpy as np
from collections import OrderedDict
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio.SeqFeature import SeqFeature, FeatureLocation
import csv
#sns.use('Agg')
from numpy import outer
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 240 )

%matplotlib inline



In [7]:
path = os.getcwd()


In [11]:
#!ls -alhtr $path
#'/data/mapping_results'
#'/data/20170111/data/TARA_metadata.csv'
#'/data'##/meta'

In [12]:
#eilat[eilat['genome_group'] == 'MGII_virus']#[['genome_size', 'genome_name']].unique()

### Load Data

In [14]:
tara  =  pd.read_csv(path + '/counts_normalization/normalized_reads.tara.20170115.tsv', delimiter='\t',low_memory=False)

eilat =  pd.read_csv(path + '/counts_normalization/normalized_reads.eilat.20170115.tsv', delimiter='\t',low_memory=False)

tara_meta =  pd.read_csv(path + '/meta/TARA_metadata.csv', delimiter=',',low_memory=False)
eilat_meta =  pd.read_csv(path + '/meta/eilat_metadata.20170111.tsv', delimiter=',',low_memory=False)


tara = tara.rename(columns = {'dataset':'RUN'})

eilat = eilat.rename(columns = {'dataset':'RUN'})

##Make Eilat DF compatible to the TARA one
eilat['Station'] = 'Station A'
#eilat['Depth'] = 5
eilat['Protocol Label'] = 'Na'
#eilat['Fraction'] = 'Na'
#eilat['LONGITUDE'] = 29.53
#eilat['LATITUDE'] = 34.97

tara_meta = tara_meta.rename(columns = {'ENA-RUN':'RUN'})
meta_run_region = tara_meta[['RUN', 'Region']]

##Add region column to the Eilat DF
eilat_region = pd.DataFrame({'Region': ['RS', 'RS', 'RS', 'RS','RS', 'RS',
                            'RS', 'RS','RS','RS','RS','RS','RS','RS','RS','RS'], 'RUN' : ['ER06',
                            'ER12','ER18','ER24','EG06','EG12',
                            'EG18', 'EG24', 'EV06', 'EV12','EV18','EV24', 
                            'Eilat_A_6', 'Eilat_A_12', 'Eilat_A_18', 'Eilat_A_24']})
eilat_region.index.rename('RUN', inplace=True)

eilat_ENA_sample = pd.DataFrame({'ENA-SAMPLE': ['Eilat_A_6', 'Eilat_A_12', 'Eilat_A_18', 'Eilat_A_24',
                                           'Eilat_A_6', 'Eilat_A_12', 'Eilat_A_18', 'Eilat_A_24',
                                           'Eilat_A_6', 'Eilat_A_12', 'Eilat_A_18', 'Eilat_A_24'], 'RUN' : ['ER06',
                            'ER12','ER18','ER24','EG06','EG12',
                            'EG18', 'EG24', 'EV06', 'EV12','EV18','EV24']})
eilat_ENA_sample.index.rename('RUN', inplace=True)

## Define groups
GroupA = ['contig__156409', 'contig__154566', 'contig__155688']
GroupB1 = ['contig__154592', 'contig__154680', 'contig__154676']
GroupB2 = ['contig__155773']
GroupC = ['contig__172037', 'contig__154940', 'contig__155057']
GroupA_TARA= ['SAMEA2621151_40', 'SAMEA2621061_122', 'SAMEA2621085_65', 'SAMEA2619923_17']
GroupB2_TARA= ['SAMEA2621151_104', 'SAMEA2620879_23', 'SAMEA2620078_38']
GroupZ = ['Z1']




correct_names = {'X_contig__155773 DNApol similar to the split but is not a split' : 'contig__155773',
                 'contig__154676 Group B? Split DNA pol  Candidate tRNA Predictions in BED format:' : 'contig__154676',
                 'contig__175897 Group A? Thermosome, DNApol' : 'contig__175897', 'contig__154564 thermosome' : 'contig__154564'}

diel_stations = {'ER06':'Eilat_RNA_6','ER12':'Eilat_RNA_12','ER18':'Eilat_RNA_18','ER24' :'Eilat_RNA_24',
                'EG06' : 'Eilat_gDNA_6','EG12' : 'Eilat_gDNA_12','EG18' : 'Eilat_gDNA_18', 'EG24' : 'Eilat_gDNA_24' ,
                'EV06' : 'Eilat_vDNA_6', 'EV12' : 'Eilat_vDNA_12','EV18' : 'Eilat_vDNA_18','EV24' : 'Eilat_vDNA_24', 
                            }

###now  correct these names in the DFs

for k, v in correct_names.iteritems():
    eilat.loc[eilat.genome_name == k, 'genome_name'] = v
    tara.loc[tara.genome_name == k, 'genome_name'] = v
    
    
## get just the coulmn we need   
tara = tara[['RUN','genome_group', 'genome_hash', 'n_count',
       'Fraction', 'Station',
       'Protocol Label', 'genome_name',
       'genome_size']]
eilat = eilat[['RUN', 'Depth', 'genome_group', 'genome_hash', 'n_count',
       'Fraction', 'Station',
       'Protocol Label', 'genome_name',
       'genome_size']]

### Load Meta Data

In [15]:
tara_meta_  =  pd.read_csv(path + '/meta/TARA_metadata_less_columns.csv', delimiter=',',low_memory=False)
tara_chem_meta  =  pd.read_csv(path + '/meta/TARA_chem_metadata_less_columns.csv', delimiter=',',low_memory=False)
eilat_chem_meta  =  pd.read_csv(path + '/meta/Eilat_station_A_surface_chem_data.csv', delimiter=',',low_memory=False)


In [16]:
eilat_chem_meta['Region'] = 'RS'

#### Add Eilat meta data to TARA region data and merge them


In [17]:
meta_col = ['Station', 'LONGITUDE', 'LATITUDE', 'Depth', 'Protocol Label',
       'Fraction', 'Region', 'ENA-SAMPLE']

In [18]:
tara_ = pd.merge(tara, tara_meta_, 
                         left_on='RUN', right_on='RUN',  how='left', suffixes = ('_tara', '_mete_tara'))
eilat_ = pd.merge(eilat, eilat_ENA_sample, 
                         left_on='RUN', right_on='RUN',  how='inner')
meta_run_region_plus_eilat =  pd.concat([eilat_region, meta_run_region], axis=0,  join='outer')

tara_meta_ENA_SAMPLE = tara_meta_[['RUN', 'ENA-SAMPLE']]

tara_.set_index('RUN', inplace=True)
tara.set_index('RUN', inplace=True)

#eilat.set_index('RUN', inplace=True)
eilat_.set_index('RUN', inplace=True)
#meta_run_region_plus_eilat.set_index('RUN', inplace=True)
eilat_ENA_sample.set_index('RUN', inplace=True)

In [19]:
##conacternate eilat and TARA
mapping_counts = pd.concat([eilat_, tara_], axis=0,  join='outer')


In [25]:
mapping_counts_normalized = pd.merge(mapping_counts, meta_run_region_plus_eilat,# axis=1,  join='inner')
                          left_index=True, right_on='RUN',  how='inner')

In [26]:
mapping_counts[mapping_counts['genome_name'] == 'SAMEA2621085_75_2']

,Chlorophyll,Depth,ENA-SAMPLE,Environment (Feature),Fraction,Nitrate Sensor,Oxygen Sensor,Protocol Label,Salinity Sensor,Sampling Station,Station,Temperature,genome_group,genome_hash,genome_name,genome_size,n_count
RUN,,,,,,,,,,,,,,,,,


In [27]:
len(mapping_counts_normalized)

105627

### save

In [28]:
#pd.DataFrame(mapping_counts_normalized.columns).to_csv(path + 'columns.csv')
pd.DataFrame(mapping_counts_normalized).to_csv(path + '/DataFrames/mapping_counts_normalized_not_melted.csv')


### create sublist of stations, diveide into three lists
will be later used for plotting

In [29]:
### divide the eilat stations
for k, v in diel_stations.iteritems():
    #print k,v
    mapping_counts_normalized.loc[mapping_counts_normalized.RUN == k, 'Station'] = v

In [30]:
###Make group Z virus a group of its own

In [31]:
mapping_counts_normalized.loc[mapping_counts_normalized.genome_name == 'Z1', 'genome_group'] = 'Group_D'

In [32]:
len(mapping_counts_normalized.Station.unique())

81

In [33]:
list_of_stations = list(OrderedDict.fromkeys(mapping_counts_normalized.Station).keys())
#list_of_stations.remove(np.nan)
sorted(list_of_stations, key=str)#.sort()
# devide them evenly into 3 lists
list_of_Station_lists = [list_of_stations[i:i+27] for i in range(0, len(list_of_stations), 27)]

In [34]:
len(list_of_stations)

81

In [35]:
Station_list_1 = list_of_Station_lists[0]
Station_list_2 = list_of_Station_lists[1]
Station_list_3 = list_of_Station_lists[2]


### Melt the dataframe

In [36]:
mapping_counts_normalized.head(n=1)

,Chlorophyll,Depth,ENA-SAMPLE,Environment (Feature),Fraction,Nitrate Sensor,Oxygen Sensor,Protocol Label,Salinity Sensor,Sampling Station,Station,Temperature,genome_group,genome_hash,genome_name,genome_size,n_count,RUN,Region
4,NaN,5.0,Eilat_A_6,NaN,BACT,NaN,NaN,Na,NaN,NaN,Eilat_gDNA_6,NaN,Cyanophage,c84cc511d5c752080560170a686fbd8d,gi|326783659|ref|NC_015287.1| Synechococcus ph...,232878,2.040508,EG06,RS


In [37]:
col_list = list(mapping_counts_normalized.columns) 
#col_list = [x for x in col_list if x not in ['Station', 'LONGITUDE', 'LATITUDE', 'Depth',
#            'Protocol Label','Fraction', 'RUN', 'Region', 'ENA-SAMPLE']]

mapping_counts_normalized_melted = pd.melt(mapping_counts_normalized, id_vars=['Station', 'LONGITUDE', 'LATITUDE', 'Depth',
            'Protocol Label','Fraction', 'RUN', 'Region', 'ENA-SAMPLE', 'genome_name', 'genome_group' ], value_vars= 'n_count', value_name='val')

In [38]:
mapping_counts_normalized_melted.head()

,Station,LONGITUDE,LATITUDE,Depth,Protocol Label,Fraction,RUN,Region,ENA-SAMPLE,genome_name,genome_group,variable,val
0,Eilat_gDNA_6,NaN,NaN,5.0,Na,BACT,EG06,RS,Eilat_A_6,gi|326783659|ref|NC_015287.1| Synechococcus ph...,Cyanophage,n_count,2.040508
1,Eilat_gDNA_6,NaN,NaN,5.0,Na,BACT,EG06,RS,Eilat_A_6,gi|932822996|dbj|BBBE01000013.1| Thermococcus ...,Euryarchaeota,n_count,0.000000
2,Eilat_gDNA_6,NaN,NaN,5.0,Na,BACT,EG06,RS,Eilat_A_6,gi|932822980|dbj|BBBE01000029.1| Thermococcus ...,Euryarchaeota,n_count,0.000000
3,Eilat_gDNA_6,NaN,NaN,5.0,Na,BACT,EG06,RS,Eilat_A_6,contig__157656,MGII,n_count,2.562425
4,Eilat_gDNA_6,NaN,NaN,5.0,Na,BACT,EG06,RS,Eilat_A_6,gi|472340512|ref|NC_020847.1| Cyanophage MED4-...,Cyanophage,n_count,0.089317


##### get unique names of groups make sure we have everything

In [39]:
k = mapping_counts_normalized_melted.genome_group.values
unique_group_list = list(OrderedDict.fromkeys(k))

##### for now don't use Unclutured phages and roseobacter in the analysis

In [40]:

mapping_counts_normalized_melted_with_uncultured_phages = mapping_counts_normalized_melted.copy()
mapping_counts_normalized_melted =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] != 'Uncultured_phage') 
                                                                 & (mapping_counts_normalized_melted['genome_group'] != 'Roseobacter_phage')]

##### subsets column with uncultured phages

In [41]:

mapping_counts_normalized_melted_with_uncultured_phages_subset_1 = mapping_counts_normalized_melted_with_uncultured_phages[mapping_counts_normalized_melted_with_uncultured_phages['Station'].isin(Station_list_1)]
mapping_counts_normalized_melted_with_uncultured_phages_subset_2 = mapping_counts_normalized_melted_with_uncultured_phages[mapping_counts_normalized_melted_with_uncultured_phages['Station'].isin(Station_list_2)]
mapping_counts_normalized_melted_with_uncultured_phages_subset_3 = mapping_counts_normalized_melted_with_uncultured_phages[mapping_counts_normalized_melted_with_uncultured_phages['Station'].isin(Station_list_3)]



##### without uncultured phages



In [42]:
mapping_counts_normalized_melted_subset_1 =  mapping_counts_normalized_melted_with_uncultured_phages_subset_1[(mapping_counts_normalized_melted_with_uncultured_phages_subset_1['genome_group'] != 'Uncultured_phage') 
                                                                 & (mapping_counts_normalized_melted_with_uncultured_phages_subset_1['genome_group'] != 'Roseobacter_phage')]
mapping_counts_normalized_melted_subset_2 =  mapping_counts_normalized_melted_with_uncultured_phages_subset_2[(mapping_counts_normalized_melted_with_uncultured_phages_subset_2['genome_group'] != 'Uncultured_phage') 
                                                                 & (mapping_counts_normalized_melted_with_uncultured_phages_subset_2['genome_group'] != 'Roseobacter_phage')]
mapping_counts_normalized_melted_subset_3 =  mapping_counts_normalized_melted_with_uncultured_phages_subset_3[(mapping_counts_normalized_melted_with_uncultured_phages_subset_3['genome_group'] != 'Uncultured_phage') 
                                                                 & (mapping_counts_normalized_melted_with_uncultured_phages_subset_3['genome_group'] != 'Roseobacter_phage')]

#### Add RNA fraction identifier (VIRUS or BACT) to Eilat samples



In [43]:
mapping_counts_normalized_melted.loc[mapping_counts_normalized_melted.Station.str.contains('Eilat_RNA'), 'Fraction'] = 'RNA'

In [44]:
#.head(1)
mapping_counts_normalized_melted.genome_group.unique()

array(['Cyanophage', 'Euryarchaeota', 'MGII', 'Prochlorococcus',
       'MGII_virus', 'SAR11', 'SAR116', 'Synechococcus', 'Crenarchaeota',
       'Halovirus', 'SAR116_Phage', 'SAR11_Phage', 'Group_D'], dtype=object)

## Save DFs

In [45]:
#save
mapping_counts_normalized_melted.to_csv(path + '/DataFrames/mapping_counts_normalized_melted.csv')

mapping_counts_normalized_melted_subset_1.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_subset_1.csv')
mapping_counts_normalized_melted_subset_2.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_subset_2.csv')
mapping_counts_normalized_melted_subset_3.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_subset_3.csv')

In [46]:
#make sure no uncultured phages are left in the mapping_counts_normalized_melted DF
mapping_counts_normalized_melted[mapping_counts_normalized_melted['genome_group'] == 'uncultured_phage'].head()


,Station,LONGITUDE,LATITUDE,Depth,Protocol Label,Fraction,RUN,Region,ENA-SAMPLE,genome_name,genome_group,variable,val


###  Select pairs of host/virus for x/y plots and create DFs

In [47]:
#'Euryarchaeota', 'MGII', 'Cyanophage', 'MGII_virus', 'SAR116_phage','Prochlorococcus','Synechococcus', 'Uncultured_phage', 'SAR11', 'Crenarchaeota', 
 #'SAR11_Phage', 'Halovirus', 'SAR116', 'Roseobacter_phage']

mapping_counts_normalized_melted_MGII_MGII_Virus =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'MGII_virus') 
                                                                 | (mapping_counts_normalized_melted['genome_group'] == 'MGII')]
mapping_counts_normalized_melted_SAR11_SAR11_phage =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'SAR11') 
                                                                 | (mapping_counts_normalized_melted['genome_group'] == 'SAR11_Phage')]
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'Synechococcus') 
                                                                 | (mapping_counts_normalized_melted['genome_group'] == 'Cyanophage') 
                                                                | (mapping_counts_normalized_melted['genome_group'] == 'Prochlorococcus')]

mapping_counts_normalized_melted_Cyanobacteria_MGII =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'Synechococcus') 
                                                                 | (mapping_counts_normalized_melted['genome_group'] == 'MGII') 
                                                                | (mapping_counts_normalized_melted['genome_group'] == 'Prochlorococcus')]
mapping_counts_normalized_melted_Cyanobacteria_SAR11 =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'Synechococcus') 
                                                                 | (mapping_counts_normalized_melted['genome_group'] == 'SAR11') 
                                                                | (mapping_counts_normalized_melted['genome_group'] == 'Prochlorococcus')]

#### Save

In [48]:

mapping_counts_normalized_melted_SAR11_SAR11_phage.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_SAR11_SAR11_phage.csv')
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.csv')
mapping_counts_normalized_melted_MGII_MGII_Virus.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_MGII_Virus.csv')
mapping_counts_normalized_melted_Cyanobacteria_SAR11.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_Cyanobacteria_SAR11.csv')
mapping_counts_normalized_melted_Cyanobacteria_MGII.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_Cyanobacteria_MGII.csv')

#### get unique names of fraction


In [49]:

k = mapping_counts_normalized_melted.Fraction


list(OrderedDict.fromkeys(k))

['BACT', 'RNA', 'VIRUS']

### mapping pivoted by phage/host plus station region and fraction.
will be used later for x/y plots

In [50]:



mapping_counts_normalized_melted_MGII_MGII_Virus_plus_fraction =\
        mapping_counts_normalized_melted_MGII_MGII_Virus.pivot_table(values=['val'],
        index=['Station', 'Region', 'Fraction'], columns=['genome_group'], aggfunc= np.sum)
mapping_counts_normalized_melted_SAR11_SAR11_phage_plus_fraction =\
        mapping_counts_normalized_melted_SAR11_SAR11_phage.pivot_table(values=['val'],
        index=['Station', 'Region', 'Fraction'], columns=['genome_group'], aggfunc= np.sum)
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction =\
        mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.pivot_table(values=['val'],
        index=['Station', 'Region', 'Fraction'], columns=['genome_group'], aggfunc= np.sum)
    


### mapping pivoted by phage/host plus station and region



In [51]:
mapping_counts_normalized_melted_MGII_MGII_Virus =\
        mapping_counts_normalized_melted_MGII_MGII_Virus.pivot_table(values=['val'],
        index=['Station', 'Region'], columns=['genome_group'], aggfunc= np.sum)
mapping_counts_normalized_melted_SAR11_SAR11_phage =\
        mapping_counts_normalized_melted_SAR11_SAR11_phage.pivot_table(values=['val'],
        index=['Station', 'Region'], columns=['genome_group'], aggfunc= np.sum)
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage =\
        mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.pivot_table(values=['val'],
        index=['Station', 'Region'], columns=['genome_group'], aggfunc= np.sum)

### remove multiindex column

In [52]:
mapping_counts_normalized_melted_MGII_MGII_Virus.columns = mapping_counts_normalized_melted_MGII_MGII_Virus.columns.get_level_values(1)
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.columns = mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.columns.get_level_values(1)
mapping_counts_normalized_melted_SAR11_SAR11_phage.columns = mapping_counts_normalized_melted_SAR11_SAR11_phage.columns.get_level_values(1)

mapping_counts_normalized_melted_MGII_MGII_Virus_plus_fraction.columns = mapping_counts_normalized_melted_MGII_MGII_Virus_plus_fraction.columns.get_level_values(1)
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction.columns = mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction.columns.get_level_values(1)
mapping_counts_normalized_melted_SAR11_SAR11_phage_plus_fraction.columns = mapping_counts_normalized_melted_SAR11_SAR11_phage_plus_fraction.columns.get_level_values(1)


### create a cyanobacteria column

In [53]:
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction['Cyanobacteria'] = mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction['Prochlorococcus'] +\
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction['Synechococcus']


#### save

In [54]:

mapping_counts_normalized_melted_MGII_MGII_Virus.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_MGII_Virus.csv')
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_Cyanobacteria_Cyanophage.csv')
mapping_counts_normalized_melted_SAR11_SAR11_phage.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_SAR11_SAR11_phage.csv')

mapping_counts_normalized_melted_MGII_MGII_Virus_plus_fraction.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_MGII_Virus_plus_fraction.csv')
mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_Cyanobacteria_Cyanophage_plus_fraction.csv')
mapping_counts_normalized_melted_SAR11_SAR11_phage_plus_fraction.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_SAR11_SAR11_phage_plus_fraction.csv')






## create a melted DF with now no contigs from tara for the chord diagram 
circ = circadian (the Eilat project)

In [55]:


mapping_counts_normalized_melted_MGII_virus_only_circ_contigs = \
                            mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'MGII_virus')
                            #& (mapping_counts_normalized_melted['genome'].isin(n_contigs))
                            & (mapping_counts_normalized_melted['Fraction'] != 'RNA')]


#### reate a groupby datdframe with sum per region and percent of contig in each region



In [56]:

MGII_virus_RS = mapping_counts_normalized_melted_MGII_virus_only_circ_contigs\
[mapping_counts_normalized_melted_MGII_virus_only_circ_contigs['Region']=='RS']
#a['p'] = (a['val'] / a.val.sum())* 100
MGII_virus_RS_genome_grouped = MGII_virus_RS.groupby('genome_name')
MGII_virus_RS_genome_grouped_sumed = MGII_virus_RS_genome_grouped.aggregate(np.sum)

MGII_virus_RS_genome_grouped_sumed['percernt'] = MGII_virus_RS_genome_grouped_sumed['val']\
                                    / MGII_virus_RS_genome_grouped_sumed.val.sum() *100

## Pivot and SUM



In [57]:

mapping_counts_normalized_melted_MGII_virus_only_circ_contigs_pivoted = mapping_counts_normalized_melted_MGII_virus_only_circ_contigs.pivot_table(index='Station', columns='genome_name', values='val')


In [58]:
mapping_counts_normalized_melted_MGII_virus_only_circ_contigs_pivoted_by_region = mapping_counts_normalized_melted_MGII_virus_only_circ_contigs.pivot_table(index='Region', columns='genome_name', values='val', 
                                                                              aggfunc='sum')




### Save

In [59]:
mapping_counts_normalized_melted_MGII_virus_only_circ_contigs_pivoted_by_region.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_virus_only_circ_contigs_pivoted_by_region.csv')


In [60]:
mapping_counts_normalized_melted_MGII_virus_only_circ_contigs_pivoted.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_virus_only_circ_contigs_pivoted.csv')


### Pivot BY GROUP

###### mapping_counts_normalized_melted_MGII_MGII_Virus_raw

In [61]:

mapping_counts_normalized_melted_MGII_MGII_Virus_raw =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'MGII_virus') 
                                                                 & (mapping_counts_normalized_melted['genome_group'] != 'MGII')
                                                                 & (mapping_counts_normalized_melted['Fraction'] != 'RNA')
                                                                 ]

mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_station = mapping_counts_normalized_melted_MGII_MGII_Virus_raw.pivot_table(index='Station', columns='genome_name', values='val', 
                                                                              aggfunc='sum')



## just vDNA

mapping_counts_normalized_melted_MGII_MGII_Virus_raw_viral_fraction =  mapping_counts_normalized_melted[(mapping_counts_normalized_melted['genome_group'] == 'MGII_virus') 
                                                                 & (mapping_counts_normalized_melted['genome_group'] != 'MGII')
                                                                 & (mapping_counts_normalized_melted['Fraction'] == 'VIRUS')
                                                                 ]

mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_station_viral_fraction = mapping_counts_normalized_melted_MGII_MGII_Virus_raw_viral_fraction.pivot_table(index='Station', columns='genome_name', values='val', 
                                                                              aggfunc='sum')



mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_ENA_SAMPLE_viral_fraction = mapping_counts_normalized_melted_MGII_MGII_Virus_raw_viral_fraction.pivot_table(index='ENA-SAMPLE', columns='genome_name', values='val', 
                                                                              aggfunc='sum')



### Save

In [62]:
mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_station.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_station.csv')

mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_station_viral_fraction.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_station_viral_fraction.csv')

mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_ENA_SAMPLE_viral_fraction.to_csv(path + '/DataFrames/mapping_counts_normalized_melted_MGII_MGII_Virus_pivoted_by_ENA_SAMPLE_viral_fraction.csv')



## META Data


#### chem data

In [63]:
## there are cells with 99999 as input change that to NaN
tara_meta_.replace('99999.0', np.nan, inplace=True)
tara_meta_.replace('Na', np.nan, inplace=True)
eilat_chem_meta.replace('Na', np.nan, inplace=True)

In [64]:
len(tara_meta_)#.head(1)

433

In [65]:
len(tara_chem_meta)#.head(1)

243

In [66]:
#mergae
tara_chem_meta_per_run = pd.merge(tara_meta_, tara_chem_meta, on='ENA-SAMPLE', how='outer')

len(tara_chem_meta_per_run)#.head()

434

#### get just those runs that we use for the analysis

In [67]:
## get just those runs that we use for the analysis
RUN_list = mapping_counts_normalized.RUN
tara_chem_meta_per_run = tara_chem_meta_per_run[tara_chem_meta_per_run['RUN'].isin(mapping_counts_normalized.RUN)]


### Merge tara_chem_meta_per_run with meta_run_region_plus_eilat

In [68]:

tara_chem_meta_per_run = pd.merge(tara_chem_meta_per_run,meta_run_region_plus_eilat, left_on='RUN', right_on='RUN')

### add all groups and MGII levels to the metadata



In [69]:
mapping_counts_normalized_melted_MGII =  mapping_counts_normalized_melted[mapping_counts_normalized_melted['genome_group'] == 'MGII']

mapping_counts_normalized_melted_MGII_pivoted_by_RUN = mapping_counts_normalized_melted_MGII.pivot_table(index='RUN', columns='genome_group', values='val', 
                                                                              aggfunc='sum')


mapping_counts_normalized_melted_MGII =  mapping_counts_normalized_melted[mapping_counts_normalized_melted['genome_group'] == 'MGII']

mapping_counts_normalized_melted_MGII_pivoted_by_RUN = mapping_counts_normalized_melted_MGII.pivot_table(index='RUN', columns='genome_group', values='val', 
                                                                              aggfunc='sum')


mapping_counts_normalized_melted_pivoted_by_RUN = mapping_counts_normalized_melted.pivot_table(index='RUN', columns='genome_group', values='val', 
                                                                              aggfunc='sum')



tara_and_eilat_chem_meta_per_run = pd.concat([eilat_chem_meta, tara_chem_meta_per_run], axis=0,  join='outer')

tara_and_eilat_chem_meta_per_run.set_index('RUN', inplace=True)


tara_and_eilat_chem_meta__plus_MGII_per_run = pd.merge(tara_and_eilat_chem_meta_per_run, mapping_counts_normalized_melted_MGII_pivoted_by_RUN, 
                                                       left_index=True, right_index=True)
tara_and_eilat_chem_meta__plus_group_counts_per_run = pd.merge(tara_and_eilat_chem_meta_per_run, mapping_counts_normalized_melted_pivoted_by_RUN, 
                                                       left_index=True, right_index=True)

In [70]:
mapping_counts_normalized_melted_pivoted_by_RUN.head()#[mapping_counts_normalized_melted_pivoted_by_RUN.Region == 'RS']

genome_group,Crenarchaeota,Cyanophage,Euryarchaeota,Group_D,Halovirus,MGII,MGII_virus,Prochlorococcus,SAR11,SAR116,SAR116_Phage,SAR11_Phage,Synechococcus
RUN,,,,,,,,,,,,,
EG06,0.000514,64.572453,0.002337,0.130245,0.0,116.074892,1.535563,89.789823,15.622486,0.471804,0.019670,0.579593,4.074782
EG12,0.000544,66.105517,0.000132,0.101129,0.0,168.318963,1.106159,73.403964,24.171924,0.278998,0.020450,0.506806,2.441884
EG18,0.000274,19.933363,0.000434,0.064064,0.0,105.268375,0.875181,107.788522,20.734830,0.272665,0.010593,0.415265,4.580567
EG24,0.000404,19.685895,0.000973,0.087274,0.0,66.311378,1.361046,141.915302,18.077407,0.175113,0.011160,0.411259,6.837379
ER06,0.177359,6.231580,0.080563,0.002640,0.0,146.401669,0.016443,110.278631,72.777144,91.181729,0.001318,0.059044,487.231304


### save

In [71]:
##save including MGII
tara_and_eilat_chem_meta_per_run.to_csv(path + '/DataFrames/tara_and_eilat_chem_meta__plus_MGII_per_run.csv')

### Save all groups
tara_and_eilat_chem_meta__plus_group_counts_per_run.to_csv(path + '/DataFrames/tara_and_eilat_chem_meta__plus_group_counts_per_run.csv')




### GroupBy Sample

In [72]:
tara_and_eilat_chem_meta__plus_MGII_grouped_by_ENA_SAMPLE = tara_and_eilat_chem_meta__plus_MGII_per_run.groupby('ENA-SAMPLE')
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE = tara_and_eilat_chem_meta__plus_group_counts_per_run.groupby('ENA-SAMPLE')

#### change 'NO2 [umol/L]' column type tot float

In [73]:
tara_and_eilat_chem_meta__plus_MGII_per_run['NO2 [umol/L]'] = tara_and_eilat_chem_meta__plus_MGII_per_run['NO2 [umol/L]'].astype(float)
tara_and_eilat_chem_meta__plus_MGII_per_run['NO2NO3 [umol/L]'] = tara_and_eilat_chem_meta__plus_MGII_per_run['NO2NO3 [umol/L]'].astype(float)


tara_and_eilat_chem_meta__plus_group_counts_per_run['NO2 [umol/L]'] = tara_and_eilat_chem_meta__plus_group_counts_per_run['NO2 [umol/L]'].astype(float)
tara_and_eilat_chem_meta__plus_group_counts_per_run['NO2NO3 [umol/L]'] = tara_and_eilat_chem_meta__plus_group_counts_per_run['NO2NO3 [umol/L]'].astype(float)



### Aggregate and calculate means

In [74]:
tara_and_eilat_chem_meta__plus_MGII_grouped_by_ENA_SAMPLE_sumed = \
tara_and_eilat_chem_meta__plus_MGII_grouped_by_ENA_SAMPLE.aggregate({'MGII': np.sum, 'Chlorophyll' : np.mean, 'Depth': np.mean,
                                                                    'NO2 [umol/L]': np.mean, 'NO2NO3 [umol/L]' : np.mean ,
                                                                    'Nitrates [umol/L]': np.mean, 'Oxygen [umol/kg]' : np.mean,
                                                                   'PO4 [umol/L]' : np.mean,
                                                                   'Region' : (lambda x: x[0]), 'Salinity [PSU]' : np.mean,
                                                                    'Temperature' : np.mean})

tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed = \
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE.aggregate({'MGII': np.sum, 'Chlorophyll' : np.mean, 'Depth': np.mean,
                                                                    'NO2 [umol/L]': np.mean, 'NO2NO3 [umol/L]' : np.mean ,
                                                                    'Nitrates [umol/L]': np.mean, 'Oxygen [umol/kg]' : np.mean,
                                                                   'PO4 [umol/L]' : np.mean,
                                                                   'Region' : (lambda x: x[0]), 'Salinity [PSU]' : np.mean,
                                                                    'Temperature' : np.mean, 'SAR11': np.sum,
                                                                    'SAR116': np.sum, 'Cyanophage' : np.sum,
                                                                     'Crenarchaeota' : np.sum, 'Euryarchaeota' : np.sum,
                                                                      'Prochlorococcus' : np.sum, 'Synechococcus' : np.sum,
                                                                      'MGII_virus' : np.sum})
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed['Cyanobacteria'] = \
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed['Prochlorococcus'] +\
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed['Synechococcus']


#### Save

In [75]:
tara_and_eilat_chem_meta__plus_MGII_grouped_by_ENA_SAMPLE_sumed.to_csv(path + '/DataFrames/tara_and_eilat_chem_meta__plus_MGII_grouped_by_ENA_SAMPLE_sumed.csv')
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed.to_csv(path + '/DataFrames/tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed.csv')




In [76]:
tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed[tara_and_eilat_chem_meta__plus_groups_grouped_by_ENA_SAMPLE_sumed.Region=='RS']

,Chlorophyll,Temperature,NO2 [umol/L],Oxygen [umol/kg],PO4 [umol/L],MGII_virus,Cyanophage,Depth,Salinity [PSU],SAR116,Nitrates [umol/L],MGII,SAR11,NO2NO3 [umol/L],Prochlorococcus,Synechococcus,Region,Euryarchaeota,Crenarchaeota,Cyanobacteria
ENA-SAMPLE,,,,,,,,,,,,,,,,,,,,
ERS488545,0.048898,25.062100,0.100,179.9,0.590,1.054436,319.635701,5.0,36.1,2.067187,2.30,188.310390,100.443014,2.80,443.083410,22.117887,RS,0.008048,0.012034,465.201296
ERS488558,0.048898,25.062100,0.100,179.9,0.590,19.312100,502.636386,5.0,36.1,1.027008,2.30,2.290923,3.449219,2.80,2.053602,1.040395,RS,0.001974,0.000105,3.093997
ERS488569,0.000000,25.824383,0.020,38.1,2.430,0.786729,1104.979957,5.0,34.6,1.926588,42.50,229.422196,164.667086,34.63,628.470519,16.924147,RS,0.006772,0.010431,645.394666
ERS488589,0.000000,25.824383,0.020,38.1,2.430,11.262284,399.614131,5.0,34.6,0.029397,42.50,0.372121,0.245717,34.63,0.590169,0.104254,RS,0.000000,0.000048,0.694423
ERS488599,0.224444,26.050367,0.100,179.9,0.590,0.928990,49.052770,80.0,36.1,1.319652,2.30,125.890022,64.280586,2.80,52.870431,4.957826,RS,0.040480,1.102711,57.828257
ERS488613,0.224444,26.050367,0.020,38.1,2.430,1.138710,10.252227,80.0,34.6,0.643579,42.50,0.719501,2.300096,34.63,5.036611,1.734996,RS,0.001442,0.006383,6.771607
ERS488621,0.163228,27.329517,0.020,38.1,2.430,0.565288,294.651724,5.0,34.6,5.289339,42.50,123.735710,212.424628,34.63,149.071197,105.207276,RS,0.014949,0.097506,254.278473
ERS488649,0.186697,27.598650,0.120,186.2,0.570,0.856313,382.135480,5.0,35.4,3.350502,4.00,202.770705,208.338262,5.46,86.081487,63.529908,RS,0.010961,0.132826,149.611395
ERS488658,0.186697,27.598650,0.120,186.2,0.570,11.841165,551.832676,5.0,35.4,0.073402,4.00,7.786986,7.346537,5.46,2.181902,1.631834,RS,0.002093,0.005690,3.813736
